In [1]:
import requests
from xml.etree import ElementTree

In [178]:
ns = {'wfs': 'http://www.opengis.net/wfs',
      'er': 'urn:cgi:xmlns:GGIC:EarthResource:1.1',
     'gml':'http://www.opengis.net/gml',
     'mo': 'http://xmlns.geoscience.gov.au/minoccml/1.0',
     'gsml': 'urn:cgi:xmlns:CGI:GeoSciML:2.0'}

In [3]:
services = {'wa' : {
                'endpoint': 'http://geossdi.dmp.wa.gov.au/services/wfs',
                'feature_type' : 'gsml:MappedFeature',
                'er_version' : 1,
                'lite_version': 1,
            }, 'nt': {
                'endpoint' : 'http://geology.data.nt.gov.au:80/geoserver/wfs',
                'feature_type' : 'gsml:MappedFeature',
                'er_version' : 1,
                'lite_version': 1,
            #},  'vic': {
            #    'endpoint': 'http://geology.data.vic.gov.au/nvcl/wfs',
            #    'feature_type' : 'gsml:MappedFeature',
            #    'er_version' : 1,
            #    'lite_version': 1,
            },  'sa': {
                'endpoint': 'https://sarigdata.pir.sa.gov.au/nvcl/geoserver/wfs',
                'feature_type' : 'gsml:MappedFeature',
                'lite_version': 2,
                'er_version' : 1
            },  'nsw': {
                'endpoint': 'https://gs.geoscience.nsw.gov.au/geoserver/wfs',
                'feature_type' : 'gsml:MappedFeature',
                'lite_version': 1,
                'er_version' : 1
            },  'tas': {
                'endpoint': 'http://www.mrt.tas.gov.au:80/web-services/wfs',
                'feature_type' : 'er:MineralOccurrence',
                'lite_version': 1,
                'er_version' : 2
            }
           }

In [116]:
feature_types = {'er:MineralOccurrence' : {
            'occurrence' : 'er:MineralOccurrence',
            'name' : './/gml:name',
            'shape' : 'gsml:occurrence/gsml:MappedFeature/gsml:shape/gml:Point/gml:pos',
            'commodities' : 'er:commodityDescription/er:Commodity'
        }, 'gsml:MappedFeature' : {
            'occurrence' : 'gsml:MappedFeature',
            'name' : './/gml:name',
            'shape' : 'gsml:shape/gml:Point/gml:pos',
            'commodities' : 'gsml:specification/er:MineralOccurrence/er:commodityDescription/er:Commodity'
        }}

In [189]:
def process_endpoints(state, service):
    endpoint = service['endpoint']
    feature_type = service['feature_type']
    parameters = {'service': 'WFS',
              'version' : '1.1.0',
              'request' : 'GetFeature',
              'typeName' : feature_type,
              'maxFeatures' : 1000}
    
    
    if service['er_version'] == 2:
        ns['er'] = 'http://xmlns.earthresourceml.org/EarthResource/2.0'
        ns['gsml'] = 'http://xmlns.geosciml.org/GeoSciML-Core/3.2'
        ns['gml'] = 'http://www.opengis.net/gml/3.2'
        ns['wfs'] = 'http://www.opengis.net/wfs/2.0'
        parameters['outputFormat'] ='gml32'
    else:
        ns['er'] = 'urn:cgi:xmlns:GGIC:EarthResource:1.1'
        ns['gsml'] = 'urn:cgi:xmlns:CGI:GeoSciML:2.0'
        ns['gml'] = 'http://www.opengis.net/gml'
        ns['wfs'] = 'http://www.opengis.net/wfs'
        parameters['outputFormat'] ='gml3'
    print(endpoint)
    response=requests.get(endpoint,params=parameters)
    root = ElementTree.fromstring(response.text)
    features = root.findall('gml:featureMembers', ns) or root.findall('gml:featureMember', ns) or root.findall('wfs:member', ns)
    for feature in features:
        process_features(feature, feature_type)

In [ ]:
def get_count(endpoint,feature_type):
    parameters = {'service': 'WFS',
              'version' : '1.1.0',
              'request' : 'GetFeature',
              'typeName' : feature_type,
              'resultType' : 'hits'
              }
    

In [190]:
def process_features(feature, feature_type):
    #print(feature)
    feature_data = []
    occurrence_path = feature_types[feature_type]['occurrence']
    occurrences = feature.findall(occurrence_path,ns)
    for occurrence in occurrences:
        feature_data.append(process_occurrences(occurrence, feature_type))
    print(feature_data)

In [194]:
def process_occurrences(occurrence, feature_type):
    name_path = feature_types[feature_type]['name']
    shape_path = feature_types[feature_type]['shape']
    commods_path=feature_types[feature_type]['commodities']
    
    
    #print(name_path, shape_path, commods_path)
    names = {name.text for name in occurrence.findall(name_path, ns) if 'codeSpace' not in name.attrib or
             ('codeSpace' in name.attrib and name.attrib['codeSpace'] != "http://www.ietf.org/rfc/rfc2616")}
    #print(names)
    shapes = occurrence.find(shape_path, ns)
    #print(shapes.text)
    commodities = occurrence.findall(commods_path, ns)
 
    commods = process_commodities(commodities)
    print(occurrence.attrib)
    print(occurrence)
    gml_id = list(occurrence.attrib.values())[0]
    
    return {'gml_id' : gml_id,
            'name' : names,
            'shape' : shapes.text,
            'commodities' : commods}

In [195]:
def process_commodities(commodities):
    commods = []
    for commodity in commodities:
        commodname = commodity.find('er:commodityName',ns)
        commod =  commodity.find('er:commodity',ns)
        if commodname is not None: 
            code = commodname.text.replace('urn:cgi:classifier:GA:commodity:','')
        elif commod is not None:
            code = commod.attrib['href']
        order = commodity.find('er:commodityRank',ns)
        importance = commodity.find('er:commodityImportance',ns)
        if order is not None:
            order = int(order.text)
        if importance is not None:
            importance = importance.text
        commods.append((code,order,importance))
    return commods

In [196]:
process_endpoints('tas', services['tas'])

http://www.mrt.tas.gov.au:80/web-services/wfs
{'id': 'mrt.er.mineraloccurrence.1'}
<Element '{http://xmlns.earthresourceml.org/EarthResource/2.0}MineralOccurrence' at 0x7f128a240cc8>
[{'gml_id': 'mrt.er.mineraloccurrence.1', 'name': {'South Deposit'}, 'shape': '145.181132551295 -41.5099510254563', 'commodities': [('http://resource.geosciml.org/classifier/cgi/commodity-code/iron', 1, None), ('http://resource.geosciml.org/classifier/cgi/commodity-code/magnetite', 2, None)]}]
{'id': 'mrt.er.mineraloccurrence.10'}
<Element '{http://xmlns.earthresourceml.org/EarthResource/2.0}MineralOccurrence' at 0x7f1290f27ef8>
[{'gml_id': 'mrt.er.mineraloccurrence.10', 'name': {'Heathorns Prospect 2'}, 'shape': '147.831576479618 -41.2909181656179', 'commodities': [('http://resource.geosciml.org/classifier/cgi/commodity-code/gold', 1, None)]}]
{'id': 'mrt.er.mineraloccurrence.100'}
<Element '{http://xmlns.earthresourceml.org/EarthResource/2.0}MineralOccurrence' at 0x7f1290f24f98>
[{'gml_id': 'mrt.er.miner

IndexError: list index out of range

In [ ]:
for state, service in services.items():
    process_endpoints(state, service)

In [56]:
url='http://www.mrt.tas.gov.au/web-services/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=er:MineralOccurrence&featureid=mrt.er.mineraloccurrence.10&outputFormat=gml32'

In [57]:
response = requests.get(url)

In [58]:
ElementTree.fromstring(response.text)

<Element '{http://www.opengis.net/wfs/2.0}FeatureCollection' at 0x7f12917b6548>

In [60]:
ns['er'] = 'http://xmlns.earthresourceml.org/EarthResource/2.0'
ns['gsml'] = 'http://xmlns.geosciml.org/GeoSciML-Core/3.2'
ns['gml'] = 'http://www.opengis.net/gml/3.2'
ns['wfs'] = 'http://www.opengis.net/wfs/2.0'

In [61]:
root = ElementTree.fromstring(response.text)
features = root.findall('gml:featureMembers', ns) or root.findall('gml:featureMember', ns) or root.findall('wfs:member', ns)

In [62]:
feature = features[0]

In [63]:
x=feature.find('.//gml:name', ns)

In [70]:
('codeSpace' in x.attrib and x.attrib['codeSpace'] != "http://www.ietf.org/rfc/rfc2616") or 'codeSpace' not in x.attrib

True

In [66]:
x.text

'Heathorns Prospect 2'